特徴量の重みを正常に反映しているか確認するため、オッズ系統を全て抜いて学習させる。

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import catboost as cb
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [2]:
data_dir = "F:/TARGET frontier/TXT33/"
data_name = "train"
tail = "fe7"
train_data = pd.read_csv(data_dir+data_name+tail+".csv",encoding="shift-jis",dtype={'レースID(旧)':str})

In [3]:
pd.set_option('display.max_columns', 100)
train_data.head()

,場所,芝・ダ,距離,クラス名,馬場状態,騎手コード,枠番,性別,年齢,キャリア,斤量,間隔,休み明け〜戦目,種牡馬,父タイプ名,調教師コード,前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走騎手コード,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走PCI3,前走RPCI,前PCI,前好走,前走平均1Fタイム,前走着差タイム,前走単勝オッズ,着差,レースID(旧),複勝オッズ下限,複勝オッズ上限,複勝配当,単勝オッズ,順位,頭数,場所距離芝ダ,場所距離芝ダ.1
0,2,2,1200,2,3,5212,6,2,3,13,56.0,6.0,5.0,572,8,1105,13,2,1400.0,2,2,660.0,2.0,3,54.0,37.2,33.9,59.53,58.2,59.7,0,11.93,0.2,14.6,0.0,0615581112,1.9,2.7,190.0,5.4,1,16,55,37
1,2,2,1200,2,3,5339,6,1,3,10,54.0,3.0,3.0,452,8,1002,1,2,1200.0,7,3,5339.0,7.0,3,54.0,34.5,34.6,48.80,46.9,49.7,0,11.52,0.0,2.2,0.0,0615581111,1.3,1.6,130.0,2.8,2,16,55,12
2,2,2,1200,2,3,5203,3,2,3,9,56.0,5.0,2.0,607,5,1089,3,2,1200.0,2,3,1018.0,8.0,2,56.0,34.5,34.4,50.13,48.0,50.3,0,11.48,0.4,28.6,0.2,0615581106,1.8,2.5,180.0,6.2,3,16,55,170
3,2,2,1200,2,3,1150,4,1,5,17,55.0,2.0,4.0,91,1,1134,2,2,1200.0,2,2,1150.0,6.0,2,53.0,34.1,34.4,48.17,46.3,49.1,0,11.42,0.0,19.1,0.3,0615581108,1.7,2.5,0.0,6.6,4,16,55,101
4,2,2,1200,2,3,1004,8,1,3,15,54.0,2.0,4.0,325,4,1030,2,2,1200.0,2,2,1134.0,2.0,5,53.0,33.6,36.4,48.17,46.3,42.3,1,11.67,1.5,8.6,0.3,0615581116,6.3,9.7,0.0,33.9,5,16,55,101


In [6]:
#test_data
test_data = pd.read_csv(data_dir+"predict"+tail+".csv",encoding="shift-jis",dtype={'レースID(旧)':str})
test_x = test_data.drop(["着差","レースID(旧)","複勝オッズ下限","複勝オッズ上限","複勝配当","順位"],axis=1)
test_y = test_data["順位"]

In [7]:
train_x = train_data.drop(["着差","レースID(旧)","複勝オッズ下限","複勝オッズ上限","複勝配当","順位"],axis=1)
train_y = train_data["順位"]

In [6]:
#catboostのparams(ベースライン)
params = {
    'depth' : 6,                  # 木の深さ
    'learning_rate' : 0.1,       # 学習率
    'early_stopping_rounds' : 50,
    'use_best_model' : True,
    'random_seed' :42
}

In [8]:
#カテゴリカル変数になっている特徴量の列名を指定
categorical_features_indices = [
    "場所",
    "芝・ダ",
    "クラス名",
    "馬場状態",
    "騎手コード",
    "枠番",
    "性別",
    "種牡馬",
    "父タイプ名",
    "調教師コード",
    "前走場所",
    "前芝・ダ",
    "前クラス名",
    "前走馬場状態",
    "前走騎手コード",
    "前走枠番",
    "前走脚質",
    "前好走",
    "場所距離芝ダ",
    "場所距離芝ダ.1",
]

In [9]:
#これがないとエラー履くので注意
train_y.fillna(10,inplace=True) #無ければ10位に
test_y.fillna(10,inplace=True)

In [10]:
#上記のものは全て文字列またはint型である必要があり、floatや欠損値は変換しなければならない
for i in categorical_features_indices:
    train_x[i].fillna(9999,inplace=True)
    test_x[i].fillna(9999,inplace=True)
for i in categorical_features_indices:
    if train_x[i].dtype != 'int64':
        train_x[i] = train_x[i].astype('int64')
    if test_x[i].dtype != 'int64':
        test_x[i] = test_x[i].astype('int64')

In [11]:
for i in categorical_features_indices:
    print(train_x[i].dtype)

int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64


In [12]:
scores = []
histories = []
models = []
y = pd.Series()
kf = KFold(n_splits=4,shuffle=False)
for tr_idx,va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    #watchlist = [(dtrain,"train"),(dvalid,"eval")]
    evals_result = {}
    model = cb.CatBoostRegressor(iterations=10000,
                                 learning_rate=0.01,
                                 depth=6,
                                 verbose=100,
                                 use_best_model=True,
                                 early_stopping_rounds=100,
                                 one_hot_max_size=100,
                                 task_type="GPU",
                                 devices='0:1'
                                 
                                )
    
    model.fit(tr_x,
              tr_y,
              cat_features=categorical_features_indices,
              eval_set=(va_x, va_y),
              plot=False,

             )
    models.append(model)
    #histories.append(evals_result)
    va_pred = model.predict(va_x)
    va_y.fillna(0.5,inplace=True)
    score = mean_squared_error(va_y,va_pred)
    score = np.sqrt(mean_squared_error(va_y,va_pred)) #rmse
    scores.append(score)
    
    #evaluationさせる
    #全部のfoldについてyに入れておいて、
    y_pred_proba = model.predict(va_x)
    y = pd.concat([y,pd.Series(y_pred_proba)])


output = train_data
output = output.assign(ya=y.values)
output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/kfeval2.csv",index=False,encoding="shift-jis")
    
print(*scores)
print(np.mean(scores))

0:	learn: 4.4393823	test: 4.4602353	best: 4.4602353 (0)	total: 34.9ms	remaining: 5m 49s
100:	learn: 3.7022374	test: 3.7401675	best: 3.7401675 (100)	total: 3.35s	remaining: 5m 28s
200:	learn: 3.5563366	test: 3.5960070	best: 3.5960070 (200)	total: 6.67s	remaining: 5m 25s
300:	learn: 3.5171325	test: 3.5576440	best: 3.5576440 (300)	total: 9.99s	remaining: 5m 21s
400:	learn: 3.4885384	test: 3.5340335	best: 3.5340335 (400)	total: 13.5s	remaining: 5m 22s
500:	learn: 3.4486793	test: 3.5104306	best: 3.5104306 (500)	total: 16.8s	remaining: 5m 19s
600:	learn: 3.4160005	test: 3.5048210	best: 3.5044200 (571)	total: 20.2s	remaining: 5m 16s
bestTest = 3.504420008
bestIteration = 571
Shrink model to first 572 iterations.
0:	learn: 4.4370614	test: 4.4670000	best: 4.4670000 (0)	total: 32.9ms	remaining: 5m 29s
100:	learn: 3.6992952	test: 3.7377985	best: 3.7377985 (100)	total: 3.27s	remaining: 5m 20s
200:	learn: 3.5543601	test: 3.5978185	best: 3.5978185 (200)	total: 6.53s	remaining: 5m 18s
300:	learn: 3.5

In [12]:
#acc:3.4866(lr=0.01,depth=7)
#acc:3.4876 (le=0.005)
#acc:3.4871 (le=0.01,depth=7→8)
#acc:3.4869 (lr=0.01,depth=6)
#学習結果のプロット
%matplotlib inline
def plot_loss(evals_result, title):
    train_metric = evals_result['train']['rmse']
    plt.plot(train_metric, label='train rmse')
    eval_metric = evals_result['eval']['rmse']
    plt.plot(eval_metric, label='eval rmse')
    plt.grid()
    plt.legend()
    plt.title(title)
    plt.xlabel('rounds')
    plt.ylabel('rmse')
    plt.show()

In [13]:
for i in range(4):
    plot_loss(histories[i],f'Training Dataset: {i}')

IndexError: list index out of range

In [13]:
test_length = len(test_data)
predict = np.array([0.0]*test_length)
for i in range(4):
    y_pred_proba = models[i].predict(test_x)
    predict += y_pred_proba

predict /= 4

In [14]:
y = pd.DataFrame(predict,columns=["y'"])
output = test_data #ここも書き換えること

output = pd.concat([output,y],axis=1)

output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/neweval6.csv",index=False,encoding="shift-jis")

In [ ]:
fscore = model.feature_importances_()

In [ ]:
fscore = model.get_score(importance_type="total_gain")
fscore = sorted([(k,v) for k,v in fscore.items()] ,key=lambda tpl: tpl[1], reverse = True)
print(*fscore[:20])

In [ ]:
#全体を100[%]とした時の重要度
mother = sum([j for i,j in fscore])
for i,j in fscore[:25]:
    print(i,round((j/mother)*100,1))

In [ ]:
slen = 20
label = []
weight = []
for i,j in fscore[:slen]:
    label.append(i)
    weight.append(j)

plt.rcdefaults() #横棒グラフモード
fig, ax = plt.subplots()
y_pos = np.arange(slen)
ax.barh(y_pos, weight,align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(label)
ax.invert_yaxis()
ax.set_xlabel('label')
ax.set_title('xgboost特徴量')
plt.show()
    

In [ ]:
#save models
import pickle
with open('xgmachina_ver0.0.7.pickle', mode='wb') as fp:
    pickle.dump(models, fp)

In [ ]:
#import models
"""import pickle
with open('xgmachina_ver0.0.6.pickle', mode='rb') as fp:
    models = pickle.load(fp)"""